# configure_test

## Tests of module configure.py 

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
import argparse

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
import invisible_cities.core.configure as conf

In [ ]:
from invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
units = SystemOfUnits()

### Configuration, input and output fles

In [ ]:
RWF_file = os.environ['IC_DATA']      + '/electrons_40keV_z250_RWF.h5'
PMP_file = os.environ['IC_DATA']      + '/electrons_40keV_z250_PMAP.h5'
CONF_file_def = os.environ['ICDIR']   + '/config/irene.conf'
CONF_file = os.environ['IC_CONFIG']      + '/irene_e40keV.conf'

### function configure_bug

**configure_bug** corresponds to the configure.py function being fixed. It declares parameters 'n', 's' and 'p' of integer type. For some reason this does not seem to work in python3.


In [ ]:
def configure_bug(input_options=sys.argv):
    """Translate command line options to a meaningfull dictionary.

    Parameters
    ----------
    input_options : sequence of strings, optional
        Input flags and parameters. Default are command line options from
        sys.argv.

    Returns
    -------
    output_options : dictionary
        Options as key-value pairs.
    """
    program, args = input_options[0], input_options[1:]
    print(program, args)
    parser = argparse.ArgumentParser()
    
    parser.add_argument("-c", metavar="cfile", type=str,
                        help="configuration file", required=True)
    parser.add_argument("-i", metavar="ifile", type=str,
                        help="input file")
    parser.add_argument("-o", metavar="ofile", type=str,
                        help="output file")
   
    print('program will crash after this line, trying to process nevt as int')
    parser.add_argument("-n", metavar="nevt", type=int,
                        help="number of events to be processed")
    
    parser.add_argument("-s", metavar="skip", type=int, default=0,
                        help="number of events to be skipped")
    parser.add_argument("-p", metavar="print_mod", type=int,
                        help="print every this number of events")
    parser.add_argument("--runall", action="store_true",
                        help="number of events to be skipped")
    parser.add_argument("-I", action="store_true", help="print info")
    parser.add_argument("-v", action="count", help="verbosity level")

    flags, extras = parser.parse_known_args(args)
    options = conf.read_config_file(flags.c) if flags.c else {}
    
    if flags.i is not None:
        options["FILE_IN"] = flags.i
    if flags.o is not None:
        options["FILE_OUT"] = flags.o
    if flags.n is not None:
        options["NEVENTS"] = flags.n
    if flags.s is not None:
        options["SKIP"] = flags.s
    if flags.p is not None:
        options["PRINT_MOD"] = flags.p
    if flags.runall:
        options["RUN_ALL"] = flags.runall
    options["INFO"] = flags.I
    if flags.v is not None:
        options["VERBOSITY"] = 50 - min(flags.v, 4)*10


    conf.print_configuration(options)
    return options


In [ ]:
def crash_configure_bug(rwf_file, pmp_file, conf_file, n_events= 1, store_pmaps=False):
    """Run Irene."""
    print('n_events = {} type of n_events = {}'.format(n_events,type(n_events)))
    print('This will crash now')
    CFP = configure_bug(['IRENE',
                     '-c', conf_file,
                     '-i', rwf_file,
                     '-o', pmp_file,
                     '-n', n_events])


In [ ]:
crash_configure_bug(RWF_file, PMP_file, CONF_file_def, n_events= 1, store_pmaps=False)

### function configure_fix

function **configure_fix** fixes the problem declaring the int variables as str and casting the int type when passing to opts.

In [ ]:
def configure_fix(input_options=sys.argv):
    """Translate command line options to a meaningfull dictionary.

    Parameters
    ----------
    input_options : sequence of strings, optional
        Input flags and parameters. Default are command line options from
        sys.argv.

    Returns
    -------
    output_options : dictionary
        Options as key-value pairs.
    """
    print('configure')
    program, args = input_options[0], input_options[1:]
    print(program, args)
    parser = argparse.ArgumentParser()
    parser.add_argument("-c", metavar="cfile", type=str,
                        help="configuration file", required=True)
    parser.add_argument("-i", metavar="ifile", type=str,
                        help="input file")
    parser.add_argument("-o", metavar="ofile", type=str,
                        help="output file")
    parser.add_argument("-n", metavar="nevt", type=str,
                        help="number of events to be processed")
    parser.add_argument("-s", metavar="skip", type=str, default=0,
                        help="number of events to be skipped")
    parser.add_argument("-p", metavar="print_mod", type=str,
                        help="print every this number of events")
    parser.add_argument("--runall", action="store_true",
                        help="number of events to be skipped")
    parser.add_argument("-I", action="store_true", help="print info")
    parser.add_argument("-v", action="count", help="verbosity level")

    flags, extras = parser.parse_known_args(args)
    print(flags)
    options = conf.read_config_file(flags.c) if flags.c else {}
    print(options)

    if flags.i is not None:
        options["FILE_IN"] = flags.i
    if flags.o is not None:
        options["FILE_OUT"] = flags.o
    if flags.n is not None:
        options["NEVENTS"] = int(flags.n)
    if flags.s is not None:
        options["SKIP"] = int(flags.s)
    if flags.p is not None:
        options["PRINT_MOD"] = int(flags.p)
    if flags.runall:
        options["RUN_ALL"] = flags.runall
    options["INFO"] = flags.I
    if flags.v is not None:
        options["VERBOSITY"] = 50 - min(flags.v, 4)*10


    conf.print_configuration(options)
    return options



In [ ]:
def test_configure_simple():
    """Test configure function. Only input parameters"""
    
    n = 10
    s = 0
    p = 1
    CFP = configure_fix(['program_name',
                     '-c', CONF_file_def,
                     '-i', 'rwf_file',
                     '-o', 'pmp_file',
                     '-n', str(n),
                     '-s', str(s),
                     '-p', str(p)])
    
    assert CFP["FILE_IN"] == 'rwf_file'
    assert CFP["FILE_OUT"] == 'pmp_file'
    assert CFP["NEVENTS"] == 10
    assert CFP["SKIP"] == 0
    assert CFP["PRINT_MOD"] == 1
    

In [ ]:
test_configure_simple()

In [ ]:
def test_configure_fix():
    """Test fixed configure function. Read from conf file"""
    
    CFP = configure_fix(['program_name',
                     '-c', CONF_file_def])
    
    assert CFP["COMPRESSION"] == 'ZLIB4'
    assert CFP["FILE_IN"] == ('/Users/jjgomezcadenas/Projects/IC/invisible_cities/'
                          +'/database/test_data//electrons_40keV_z250_RWF.h5')
    assert CFP["FILE_OUT"] == ('/Users/jjgomezcadenas/Projects/IC/invisible_cities/'
                           +'/database/test_data//electrons_40keV_z250_PMP.h5')
    assert CFP["INFO"] ==                   False
    assert CFP["NBASELINE"] ==              28000
    assert CFP["NEVENTS"] ==                10
    assert CFP["NMAU"] ==                   100
    assert CFP["NPRINT"] ==                 1
    assert CFP["PRINT_EMPTY_EVENTS"] ==     1
    assert CFP["RUN_ALL"] ==                False
    assert CFP["RUN_NUMBER"] ==             0
    assert CFP["S1_LMAX"] ==                20
    assert CFP["S1_LMIN"] ==                6
    assert CFP["S1_STRIDE"] ==              4
    assert CFP["S1_TMAX"] ==                590
    assert CFP["S1_TMIN"] ==                10
    assert CFP["S2_LMAX"] ==                100000
    assert CFP["S2_LMIN"] ==                100
    assert CFP["S2_STRIDE"] ==              40
    assert CFP["S2_TMAX"] ==                1190
    assert CFP["S2_TMIN"] ==                0
    assert CFP["SKIP"] ==                   0
    assert CFP["THR_CSUM"] ==               0.5
    assert CFP["THR_MAU"] ==                3
    assert CFP["THR_SIPM_S2"] ==            30
    assert CFP["THR_TRIGGER"] ==            5
    assert CFP["THR_ZS"] ==                 20
    assert CFP["VERBOSITY"] ==              20

In [ ]:
test_configure_fix()

In [ ]:
def test_configure():
    """Test configure function. Read from conf file"""
    
    n = 10
    s = 0
    p = 1
    CFP = conf.configure(['program_name',
                     '-c', CONF_file_def,
                     '-i', 'rwf_file',
                     '-o', 'pmp_file',
                     '-n', str(n),
                     '-s', str(s),
                     '-p', str(p)])
    
    assert CFP["COMPRESSION"] == 'ZLIB4'
    assert CFP["FILE_IN"] == 'rwf_file'
    assert CFP["FILE_OUT"] == 'pmp_file'
    assert CFP["INFO"] ==                   False
    assert CFP["NBASELINE"] ==              28000
    assert CFP["NEVENTS"] ==                10
    assert CFP["NMAU"] ==                   100
    assert CFP["NPRINT"] ==                 1
    assert CFP["PRINT_EMPTY_EVENTS"] ==     1
    assert CFP["RUN_ALL"] ==                False
    assert CFP["RUN_NUMBER"] ==             0
    assert CFP["S1_LMAX"] ==                20
    assert CFP["S1_LMIN"] ==                6
    assert CFP["S1_STRIDE"] ==              4
    assert CFP["S1_TMAX"] ==                590
    assert CFP["S1_TMIN"] ==                10
    assert CFP["S2_LMAX"] ==                100000
    assert CFP["S2_LMIN"] ==                100
    assert CFP["S2_STRIDE"] ==              40
    assert CFP["S2_TMAX"] ==                1190
    assert CFP["S2_TMIN"] ==                0
    assert CFP["SKIP"] ==                   0
    assert CFP["THR_CSUM"] ==               0.5
    assert CFP["THR_MAU"] ==                3
    assert CFP["THR_SIPM_S2"] ==            30
    assert CFP["THR_TRIGGER"] ==            5
    assert CFP["THR_ZS"] ==                 20
    assert CFP["VERBOSITY"] ==              20

In [ ]:
test_configure()